<a href="https://colab.research.google.com/github/NjorogeGodwin/Machine-Language-Translator-model-1/blob/main/Copy_of_Best_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Check GPU availability and set device
import torch
if torch.cuda.is_available():
    print("GPU is available. Setting device to GPU.")
    device = torch.device("cuda")
else:
    print("GPU not available. Using CPU.")
    device = torch.device("cpu")

# Mount Google Drive (Optional but recommended for saving models)
from google.colab import drive
try:
    drive.mount('/content/drive')
    print("Google Drive mounted successfully.")
except Exception as e:
    print(f"Error mounting Google Drive: {e}")

In [ ]:
# Install necessary libraries
!pip install transformers[torch] datasets sacrebleu sentencepiece accelerate -U

In [ ]:
import os

# --- Ensure files are uploaded to /content/ in Colab ---
kikuyu_file_path = 'kikuyu-train-data.txt'
swahili_file_path = 'kiswahili-train-data.txt'

kikuyu_sentences = []
swahili_sentences = []
data_loaded = False

# Check if files exist
if not os.path.exists(kikuyu_file_path):
    print(f"ERROR: File not found at /content/{kikuyu_file_path}. Please upload it to the Colab session.")
if not os.path.exists(swahili_file_path):
    print(f"ERROR: File not found at /content/{swahili_file_path}. Please upload it to the Colab session.")

# Load data only if files exist
if os.path.exists(kikuyu_file_path) and os.path.exists(swahili_file_path):
    print("Loading data...")
    try:
        with open(kikuyu_file_path, 'r', encoding='utf-8') as f_ki, \
             open(swahili_file_path, 'r', encoding='utf-8') as f_sw:
            for line_no, (line_ki, line_sw) in enumerate(zip(f_ki, f_sw)):
                line_ki = line_ki.strip()
                line_sw = line_sw.strip()
                if line_ki and line_sw: # Skip pairs with empty lines
                    kikuyu_sentences.append(line_ki)
                    swahili_sentences.append(line_sw)
                elif line_ki or line_sw: # Warn about partially empty lines if needed
                    print(f"Warning: Partially empty line pair at line number {line_no + 1}. Skipping.")

        if not kikuyu_sentences:
             print("Warning: No valid sentence pairs found after loading.")
        else:
            print(f"Loaded {len(kikuyu_sentences)} sentence pairs.")
            data_loaded = True

            # Inspect the first 5 pairs
            print("\n--- First 5 Sentence Pairs ---")
            for i in range(min(5, len(kikuyu_sentences))):
                 print(f"KI: {kikuyu_sentences[i]}")
                 print(f"SW: {swahili_sentences[i]}\n")
    except Exception as e:
        print(f"An error occurred while reading the files: {e}")
        data_loaded = False # Ensure flag is false on error
else:
    print("Cannot proceed without the data files.")



In [ ]:
import re

def normalize_text(text):
  """Basic text normalization: lowercase and remove extra spaces."""
  text = text.lower()
  text = re.sub(r'\s+', ' ', text).strip()
  # Add more specific normalization rules here if needed
  return text

if data_loaded: # Check if data was loaded successfully
    print("Normalizing text...")
    try:
        kikuyu_sentences = [normalize_text(s) for s in kikuyu_sentences]
        swahili_sentences = [normalize_text(s) for s in swahili_sentences]

        print("\n--- First 5 Normalized Pairs ---")
        for i in range(min(5, len(kikuyu_sentences))):
             print(f"KI: {kikuyu_sentences[i]}")
             print(f"SW: {swahili_sentences[i]}\n")
    except Exception as e:
        print(f"An error occurred during normalization: {e}")
        data_loaded = False # Indicate data issue
else:
    print("Skipping normalization as data was not loaded successfully.")

In [ ]:
from sklearn.model_selection import train_test_split

data_split_success = False
if data_loaded: # Check if data processing was successful so far
    print("Splitting data...")
    try:
        # Combine into pairs first
        parallel_data = list(zip(kikuyu_sentences, swahili_sentences))

        if not parallel_data:
             raise ValueError("No parallel data pairs available for splitting.")

        # Split: 80% train, 20% temporary
        train_data, temp_data = train_test_split(parallel_data, test_size=0.2, random_state=42, shuffle=True)
        # Split temporary: 50% validation, 50% test (results in 10% val, 10% test of original)
        val_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42, shuffle=True)

        print(f"Train set size: {len(train_data)}")
        print(f"Validation set size: {len(val_data)}")
        print(f"Test set size: {len(test_data)}")

        # Basic check for non-empty splits
        if not train_data or not val_data or not test_data:
            print("Warning: One or more data splits are empty after splitting.")
        else:
            data_split_success = True

    except Exception as e:
        print(f"An error occurred during data splitting: {e}")
        data_split_success = False
else:
    print("Skipping data splitting as previous steps failed.")


In [ ]:
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer
import traceback # For detailed error printing

tokenization_success = False
# Ensure data_split_success is True from Step 4
if 'data_split_success' in locals() and data_split_success:
    print("Creating Hugging Face Datasets...")
    try:
        def create_hf_dataset(data_split):
            """Converts a list of (ki, sw) pairs into a Hugging Face Dataset."""
            if not data_split:
                return Dataset.from_dict({"translation": []})
            kikuyu = [pair[0] for pair in data_split]
            swahili = [pair[1] for pair in data_split]
            # Structure for MT5: keep separate columns for easier prefixing
            return Dataset.from_dict({"kikuyu": kikuyu, "swahili": swahili})

        # Create DatasetDict
        raw_datasets = DatasetDict({
            "train": create_hf_dataset(train_data),
            "validation": create_hf_dataset(val_data),
            "test": create_hf_dataset(test_data)
        })
        print(raw_datasets)

        # --- Tokenization using a NEW Pre-trained Tokenizer ---
        print("\nLoading pre-trained tokenizer (google/mt5-small)...")
        # *** SWITCHING MODEL CHECKPOINT ***
        model_checkpoint = "google/mt5-small"

        try:
            # 1. Load the MT5 tokenizer
            tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
            print(f"MT5 tokenizer loaded. Vocab size: {tokenizer.vocab_size}")

            # MT5 uses prefixes, not src_lang/tgt_lang attributes
            # No language code verification needed here like for mBART

            # --- Proceed with tokenization mapping ---
            source_lang = "Kikuyu" # For the prefix
            target_lang = "Swahili" # For the prefix
            prefix = f"translate {source_lang} to {target_lang}: "

            source_lang_key = "kikuyu"  # Column name in the dataset
            target_lang_key = "swahili" # Column name in the dataset
            max_input_length = 128
            max_target_length = 128

            print(f"\nUsing prefix for MT5: '{prefix}'")
            print("Tokenizing datasets...")

            def preprocess_function(examples):
                """Tokenizes a batch of examples for MT5."""
                if not examples[source_lang_key]:
                    return {"input_ids": [], "attention_mask": [], "labels": []}

                # Add prefix to source sentences
                inputs = [prefix + sentence for sentence in examples[source_lang_key]]
                targets = examples[target_lang_key]

                # Tokenize inputs (source with prefix)
                model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

                # Tokenize targets (without prefix)
                # Do NOT use "with tokenizer.as_target_tokenizer()" for MT5
                labels = tokenizer(targets, max_length=max_target_length, truncation=True)

                model_inputs["labels"] = labels["input_ids"]
                return model_inputs

            # Apply tokenization to all splits
            # Ensure original columns are kept temporarily if needed, then remove
            tokenized_datasets = raw_datasets.map(
                preprocess_function,
                batched=True,
                remove_columns=raw_datasets["train"].column_names # Remove original text columns
            )

            print("\nTokenized Datasets:")
            print(tokenized_datasets)
            if not tokenized_datasets["train"] or not tokenized_datasets["validation"] or not tokenized_datasets["test"]:
                 print("Warning: One or more tokenized datasets are empty.")
            else:
                print("\nExample of tokenized input:")
                # Decode example to show prefix
                example_input_ids = tokenized_datasets["train"][0]['input_ids']
                example_labels = tokenized_datasets["train"][0]['labels']
                print(f"Decoded Input: {tokenizer.decode(example_input_ids, skip_special_tokens=True)}")
                print(f"Decoded Label: {tokenizer.decode(example_labels, skip_special_tokens=True)}")
                tokenization_success = True # Set flag on success

        except Exception as e:
            print(f"CRITICAL ERROR during tokenizer loading or processing with {model_checkpoint}: {e}")
            print(traceback.format_exc()) # Print detailed traceback
            tokenization_success = False
            if 'tokenizer' in locals(): del tokenizer
            if 'tokenized_datasets' in locals(): del tokenized_datasets

    except Exception as e:
        print(f"An error occurred during Hugging Face dataset creation: {e}")
        tokenization_success = False

else:
    print("Skipping Hugging Face dataset preparation as previous steps failed or data splitting was unsuccessful.")

# Make tokenizer available globally if successful
if tokenization_success:
    print(f"Tokenization completed successfully using {model_checkpoint}.")
    # IMPORTANT: Update subsequent steps to use the new model_checkpoint!
    print("REMINDER: Ensure Step 6 (Load Model) uses the same checkpoint:", model_checkpoint)
else:
    print("Tokenization failed. Cannot proceed to subsequent steps.")



In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments # Import Seq2SeqTrainingArguments here if needed for type hints, otherwise it's mainly used in Step 8
import torch
import traceback
import datasets # Import datasets for type hints if needed

model_loaded = False
collator_defined = False
# Ensure tokenizer is loaded from Step 4
if 'tokenizer' in locals() and tokenizer is not None:
    print("Loading MT5 model...")
    try:
        # Load the pre-trained MT5 model for sequence-to-sequence tasks
        # model_checkpoint should be "google/mt5-small" from Step 4
        if 'model_checkpoint' not in locals():
             print("ERROR: model_checkpoint variable not found. Cannot load model.")
             model_loaded = False
        else:
             model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)
             print(f"Model '{model_checkpoint}' loaded successfully.")

             # --- Check if GPU is available and move model ---
             if torch.cuda.is_available():
                 print("GPU detected. Moving model to GPU...")
                 try:
                     model.to('cuda')
                     print("Model moved to GPU.")
                 except Exception as e:
                     print(f"Warning: Failed to move model to GPU: {e}. Using CPU.")
             else:
                 print("No GPU detected. Using CPU.")
             # --- End GPU Check ---

             model_loaded = True

             print("\nDefining Data Collator...")
             # Define the data collator for sequence-to-sequence tasks
             # It handles padding inputs and labels dynamically per batch
             try:
                 # --- Revised Collator Initialization ---
                 data_collator = DataCollatorForSeq2Seq(
                     tokenizer=tokenizer, # Explicitly pass the tokenizer
                     model=model, # Pass the model for potential model-specific padding
                     label_pad_token_id=-100, # Ensure padding tokens in labels are ignored by loss function
                     pad_to_multiple_of=8 if torch.cuda.is_available() else None # Optimize for GPU if available
                 )
                 # --- End of Revision ---
                 print("Data collator ready.")
                 collator_defined = True
             except Exception as e:
                 print(f"Error defining data collator: {e}")
                 print(traceback.format_exc())
                 if 'data_collator' in locals(): del data_collator
                 collator_defined = False

    except Exception as e:
        print(f"Error loading model: {e}")
        print(traceback.format_exc())
        if 'model' in locals(): del model
        model_loaded = False
else:
    print("Skipping model loading and collator definition as tokenizer is missing.")

# Combine flags for clarity in subsequent steps if needed (optional)
# model_and_collator_ready = model_loaded and collator_defined


In [ ]:
!pip install evaluate sacrebleu -U

In [ ]:
#Step 7
import numpy as np
import evaluate # Use the evaluate library
import torch
import traceback # For detailed error printing

metrics_defined = False
# Ensure model loaded successfully and tokenizer exists
if 'model_loaded' in locals() and model_loaded and 'tokenizer' in locals():
    print("Loading BLEU metric using 'evaluate' library...")
    try:
        # Load the metric using evaluate.load
        metric = evaluate.load("sacrebleu")
        print("BLEU metric loaded successfully.")

        def postprocess_text(preds, labels):
            """Helper function to clean up predictions and labels for BLEU calculation."""
            preds = [pred.strip() for pred in preds]
            # Ensure labels are a list of lists for sacrebleu
            labels = [[label.strip()] for label in labels]
            return preds, labels

        def compute_metrics(eval_preds):
            """Computes BLEU score during evaluation with added robustness and ID checks."""
            preds, labels = eval_preds
            if isinstance(preds, tuple):
                preds = preds[0] # Get the actual predictions if it's a tuple

            # Replace -100 used for padding/masking in labels with the pad_token_id
            pad_token_id = tokenizer.pad_token_id
            labels = np.where(labels != -100, labels, pad_token_id)

            # --- Added ID Range Check ---
            # Check if predicted token IDs are within the valid vocabulary range
            min_id = 0
            max_id = tokenizer.vocab_size - 1
            # Ensure preds is a numpy array for easier checking
            if not isinstance(preds, np.ndarray):
                 preds = np.array(preds) # Convert if necessary

            # Check for out-of-range values (excluding potential padding/special tokens if needed, though usually they are within range)
            # Using np.any for efficiency
            if np.any((preds < min_id) | (preds > max_id)):
                 print(f"Warning: Detected invalid token IDs in predictions (outside range [{min_id}, {max_id}]). Skipping decoding for this batch.")
                 # Return default values as decoding will fail
                 return {"bleu": 0.0, "gen_len": 0.0}
            # --- End of ID Range Check ---

            decoded_preds, decoded_labels = [], []
            # Decode generated sequences and reference sequences
            try:
                # Decode predictions (should be safe now after the check)
                decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

                # Decode labels
                decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

            except Exception as decode_err:
                # Keep this catch just in case, though the range check should prevent OverflowError
                print(f"ERROR during decoding (unexpected): {decode_err}")
                print(traceback.format_exc())
                return {"bleu": 0.0, "gen_len": 0.0}

            # Filter out empty predictions/labels
            valid_preds = []
            valid_labels = []
            for pred, label_list in zip(decoded_preds, postprocess_text([], decoded_labels)[1]):
                 label = label_list[0]
                 if pred and label:
                     valid_preds.append(pred)
                     valid_labels.append([label])

            if not valid_preds or not valid_labels:
                 # This warning might still appear if the model generates empty strings even with valid IDs
                 print("Warning: No valid prediction/label pairs found after filtering empty strings. Returning BLEU 0.")
                 result = {"bleu": 0.0}
            else:
                 processed_preds, processed_labels = postprocess_text(valid_preds, [lbl[0] for lbl in valid_labels])
                 try:
                     bleu_result = metric.compute(predictions=processed_preds, references=processed_labels)
                     result = {"bleu": bleu_result["score"]}
                 except Exception as bleu_err:
                     print(f"ERROR computing BLEU: {bleu_err}")
                     print(traceback.format_exc())
                     result = {"bleu": 0.0}

            # Calculate generated length metric
            try:
                prediction_lens = [np.count_nonzero(pred_ids != pad_token_id) for pred_ids in preds]
                result["gen_len"] = np.mean(prediction_lens) if prediction_lens else 0.0
            except Exception as len_err:
                 print(f"Error calculating gen_len: {len_err}")
                 result["gen_len"] = 0.0

            result = {k: round(v, 4) for k, v in result.items()}
            return result

        print("compute_metrics function defined (with ID range check and robustness).")
        metrics_defined = True

    except Exception as e:
        print(f"Error setting up metrics: {e}")
        print(traceback.format_exc())
        if 'metric' in locals(): del metric
        if 'compute_metrics' in locals(): del compute_metrics
        metrics_defined = False
else:
    print("Skipping metric definition as model loading failed or tokenizer is missing.")



In [ ]:
#step 8
from transformers import Seq2SeqTrainingArguments
import os
import torch # Import torch to check for GPU
import traceback # For detailed error reporting

args_defined = False
# Ensure model_loaded flag exists from Step 6
if 'model_loaded' in locals() and model_loaded:
    print("Defining Training Arguments (Balanced Approach for Stability)...")
    try:
        # Define the directory for saving checkpoints and the final model
        output_base_dir = "c:/Users/pc/Desktop/Machine Language Proj/AI Project train data/results"
        if not os.path.exists(output_base_dir):
            try:
                os.makedirs(output_base_dir)
                print(f"Created output directory: {output_base_dir}")
            except OSError as e:
                print(f"Error creating directory {output_base_dir}: {e}")
                raise e # Re-raise the error

        # Determine batch size based on GPU availability
        per_device_batch_size = 8 if torch.cuda.is_available() else 4
        gradient_accumulation_steps = 2 # Keep accumulation steps

        args = Seq2SeqTrainingArguments(
            output_dir=output_base_dir,
            # --- Evaluation and Saving Strategy ---
            eval_strategy="epoch",             # Evaluate every epoch
            save_strategy="epoch",             # Save checkpoint every epoch
            # --- Learning Rate and Optimizer ---
            learning_rate=2e-5,                # Standard fine-tuning LR
            adam_beta1=0.9,                    # Default AdamW beta1
            adam_beta2=0.999,                  # Default AdamW beta2
            adam_epsilon=1e-8,                 # Default AdamW epsilon
            weight_decay=0.01,                 # Standard weight decay
            # --- Stability Measures ---
            max_grad_norm=1.0,                 # Gradient clipping
            warmup_steps=100,                  # Learning rate warmup
            fp16=torch.cuda.is_available(),    # Re-enable FP16 if GPU available
            # --- Training Length ---
            num_train_epochs=4,                # Train for 4 epochs as requested
            # --- Batch Size ---
            per_device_train_batch_size=per_device_batch_size,
            per_device_eval_batch_size=per_device_batch_size, # Use same for eval
            gradient_accumulation_steps=gradient_accumulation_steps,
            # --- Model Loading and Metrics ---
            load_best_model_at_end=True,       # Load the best model based on metric
            metric_for_best_model="bleu",      # Use BLEU score to find best model
            predict_with_generate=True,        # Needed for BLEU calculation
            # --- Logging and Checkpoints ---
            logging_strategy="steps",          # Log periodically
            logging_steps=100,                 # Log every 100 steps
            save_total_limit=3,                # Keep only the last 3 checkpoints + best
            # --- Other ---
            push_to_hub=False,
            # report_to="tensorboard"          # Optional: Uncomment for TensorBoard
        )
        print("Training arguments defined.")
        print(f"  Effective batch size: {per_device_batch_size * gradient_accumulation_steps * (torch.cuda.device_count() if torch.cuda.is_available() else 1)}")
        print(f"  FP16 enabled: {args.fp16}")
        print(f"  Learning Rate: {args.learning_rate}")
        print(f"  Num Train Epochs: {args.num_train_epochs}")
        print(f"  Warmup Steps: {args.warmup_steps}")
        print(f"  Max Grad Norm: {args.max_grad_norm}")
        print(f"  Load Best Model at End: {args.load_best_model_at_end}")
        print(f"  Metric for Best Model: {args.metric_for_best_model}")
        args_defined = True

    except Exception as e:
        print(f"Error defining training arguments: {e}")
        print(traceback.format_exc())
        if 'args' in locals(): del args
        args_defined = False
else:
    print("Skipping training arguments definition as the model was not loaded successfully.")





In [ ]:
# Step 9: Memory-Optimized Kikuyu-Kiswahili Translation Model Training (Enhanced)
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding, Dropout
import gc
import os
import logging
import time
import pickle
import signal
from sklearn.model_selection import train_test_split

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Print TensorFlow version
logger.info(f"TensorFlow version: {tf.__version__}")

# Force CPU usage to avoid CUDA errors
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'  # Force CPU usage
logger.info("Using CPU for training (GPU disabled to prevent errors)")

# Add timeout handler to prevent hanging
class TimeoutException(Exception):
    pass

def timeout_handler(signum, frame):
    raise TimeoutException("Function timed out!")

# Step 9.1: Load data with memory constraints and improved error handling
logger.info("\n--- Loading Data ---")

# Set a limit on the number of sentences to process
MAX_SAMPLES = 1000  # Adjust based on your RAM constraints

# Function to safely load text files with error handling
def safe_load_file(file_path, encoding='utf-8'):
    try:
        with open(file_path, 'r', encoding=encoding) as file:
            lines = file.readlines()
            texts = [line.strip() for line in lines if line.strip()]
            logger.info(f"Successfully loaded {len(texts)} lines from {file_path}")
            return texts
    except FileNotFoundError:
        logger.error(f"File not found: {file_path}")
        # Try to create from CSV if available
        try:
            csv_path = file_path.replace('.txt', '.csv')
            with open(csv_path, 'r', encoding=encoding) as file:
                lines = file.readlines()
                texts = [line.strip() for line in lines if line.strip()]
                # Save as txt for future use
                with open(file_path, 'w', encoding=encoding) as out_file:
                    for text in texts:
                        out_file.write(f"{text}\n")
                logger.info(f"Created {file_path} from CSV with {len(texts)} lines")
                return texts
        except:
            logger.error(f"Could not create {file_path} from CSV")
            return []
    except UnicodeDecodeError:
        logger.error(f"Unicode decode error with {encoding}. Trying with 'latin-1'")
        # Fallback to latin-1 encoding
        return safe_load_file(file_path, 'latin-1')
    except Exception as e:
        logger.error(f"Error loading {file_path}: {e}")
        return []

# Load Kikuyu data
kikuyu_file_path = 'kikuyu-train-data.txt'
kikuyu_texts = safe_load_file(kikuyu_file_path)
if not kikuyu_texts:
    # Try to extract from CSV directly
    try:
        csv_path = 'kikuyu-train-data.csv'
        with open(csv_path, 'r', encoding='utf-8') as file:
            kikuyu_texts = [line.strip() for line in file.readlines() if line.strip()]
            logger.info(f"Extracted {len(kikuyu_texts)} lines directly from CSV")
    except Exception as e:
        logger.error(f"Failed to extract from CSV: {e}")
        raise ValueError(f"Failed to load Kikuyu data")

# Load Kiswahili data
kiswahili_file_path = 'kiswahili-train-data.txt'
kiswahili_texts = safe_load_file(kiswahili_file_path)
if not kiswahili_texts:
    raise ValueError(f"Failed to load Kiswahili data from {kiswahili_file_path}")

# Ensure we have the same number of sentences in both languages
min_len = min(len(kikuyu_texts), len(kiswahili_texts), MAX_SAMPLES)
kikuyu_texts = kikuyu_texts[:min_len]
kiswahili_texts = kiswahili_texts[:min_len]
logger.info(f"Using {min_len} parallel sentences for training")

# Print some examples to verify alignment
logger.info("\n--- Sample Sentence Pairs ---")
for i in range(min(3, min_len)):
    logger.info(f"Kikuyu: {kikuyu_texts[i]}")
    logger.info(f"Kiswahili: {kiswahili_texts[i]}")
    logger.info("")

# Step 9.2: Improved text preprocessing
logger.info("\n--- Preprocessing Text ---")

# Text normalization function
def normalize_text(texts):
    normalized = []
    for text in texts:
        # Convert to lowercase
        text = text.lower()
        # Remove extra spaces
        text = ' '.join(text.split())
        normalized.append(text)
    return normalized

# Apply normalization
kikuyu_texts = normalize_text(kikuyu_texts)
kiswahili_texts = normalize_text(kiswahili_texts)
logger.info("Text normalization completed")

# Step 9.3: Tokenize with reduced parameters and improved handling
logger.info("\n--- Tokenizing Text ---")

# Define parameters with reduced complexity
max_num_words = 5000  # Reduced vocabulary size
max_len_source = 30   # Reduced sequence length
max_len_target = 30   # Reduced sequence length
embedding_dim = 32    # Reduced embedding dimension
latent_dim = 32       # Reduced LSTM dimension

# Create tokenizers with OOV handling
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

source_tokenizer = Tokenizer(num_words=max_num_words, oov_token='<OOV>', filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')
target_tokenizer = Tokenizer(num_words=max_num_words, oov_token='<OOV>', filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')

# Fit tokenizers on texts
source_tokenizer.fit_on_texts(kikuyu_texts)
target_tokenizer.fit_on_texts(kiswahili_texts)

# Calculate vocabulary sizes
source_vocab_size = min(max_num_words, len(source_tokenizer.word_index) + 1)
target_vocab_size = min(max_num_words, len(target_tokenizer.word_index) + 1)
logger.info(f"Kikuyu vocabulary size: {source_vocab_size}")
logger.info(f"Kiswahili vocabulary size: {target_vocab_size}")

# Convert texts to sequences with error handling
try:
    source_sequences = source_tokenizer.texts_to_sequences(kikuyu_texts)
    target_sequences = target_tokenizer.texts_to_sequences(kiswahili_texts)
    logger.info("Successfully converted texts to sequences")
except Exception as e:
    logger.error(f"Error during sequence conversion: {e}")
    raise

# Pad sequences
encoder_input_data = pad_sequences(source_sequences, maxlen=max_len_source, padding='post')
decoder_input_data = pad_sequences(target_sequences, maxlen=max_len_target, padding='post')

# Create target data (shifted by one)
decoder_target_data = np.zeros_like(decoder_input_data)
decoder_target_data[:, :-1] = decoder_input_data[:, 1:]

# Clear memory
del source_sequences, target_sequences
gc.collect()
logger.info(f"Encoder input shape: {encoder_input_data.shape}")
logger.info(f"Decoder input shape: {decoder_input_data.shape}")
logger.info(f"Decoder target shape: {decoder_target_data.shape}")

# Step 9.4: Split data with smaller validation set
train_encoder_input, val_encoder_input, train_decoder_input, val_decoder_input, train_decoder_target, val_decoder_target = train_test_split(
    encoder_input_data, decoder_input_data, decoder_target_data, test_size=0.1, random_state=42
)

# Clear memory
del encoder_input_data, decoder_input_data, decoder_target_data
gc.collect()
logger.info(f"Training samples: {len(train_encoder_input)}")
logger.info(f"Validation samples: {len(val_encoder_input)}")

# Step 9.5: Create TensorFlow datasets for memory-efficient training
BATCH_SIZE = 8  # Reduced batch size

# Create datasets with prefetching for better performance
train_dataset = tf.data.Dataset.from_tensor_slices(
    ((train_encoder_input, train_decoder_input), train_decoder_target)
).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

val_dataset = tf.data.Dataset.from_tensor_slices(
    ((val_encoder_input, val_decoder_input), val_decoder_target)
).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

# Clear memory
del train_encoder_input, train_decoder_input, train_decoder_target
del val_encoder_input, val_decoder_input, val_decoder_target
gc.collect()

# Step 9.6: Define a simpler model architecture to avoid shape issues
logger.info("\n--- Defining Model Architecture ---")

# Encoder
encoder_inputs = Input(shape=(max_len_source,), name='encoder_inputs')
encoder_embedding = Embedding(input_dim=source_vocab_size, output_dim=embedding_dim, name='encoder_embedding')(encoder_inputs)

# Use a simpler LSTM configuration to avoid shape issues
encoder_lstm = LSTM(latent_dim, return_state=True, name='encoder_lstm')
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = Input(shape=(max_len_target,), name='decoder_inputs')
decoder_embedding = Embedding(input_dim=target_vocab_size, output_dim=embedding_dim, name='decoder_embedding')(decoder_inputs)

# Use a simpler LSTM configuration to avoid shape issues
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True, name='decoder_lstm')
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)

# Add dropout for regularization
decoder_outputs = Dropout(0.2)(decoder_outputs)

# Add a dense layer with softmax activation
decoder_dense = Dense(target_vocab_size, activation='softmax', name='decoder_dense')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
logger.info("Model architecture defined")

# Step 9.7: Compile the model with gradient clipping to prevent exploding gradients
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001, clipnorm=1.0)
model.compile(
    optimizer=optimizer,
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Print model summary
model.summary()

# Step 9.8: Define callbacks for better training
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

checkpoint_path = "best_model.keras"  # Using .keras extension as recommended
callbacks = [
    # Early stopping to prevent overfitting
    EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True,
        verbose=1
    ),
    # Save the best model using the recommended format
    ModelCheckpoint(
        filepath=checkpoint_path,
        save_best_only=True,
        monitor='val_loss',
        verbose=1
    ),
    # Reduce learning rate when learning plateaus
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=2,
        verbose=1,
        min_lr=0.0001
    )
]

# Step 9.9: Train the model with error handling and timeout
logger.info("\n--- Training Model ---")
try:
    # Set a timeout for training (30 minutes)
    signal.signal(signal.SIGALRM, timeout_handler)
    signal.alarm(1800)  # 30 minutes timeout

    # Wrap training in a try-except block to handle errors gracefully
    start_time = time.time()

    history = model.fit(
        train_dataset,
        epochs=10,  # Reduced number of epochs to avoid long training times
        validation_data=val_dataset,
        callbacks=callbacks,
        verbose=1
    )

    # Cancel the timeout
    signal.alarm(0)

    training_time = time.time() - start_time
    logger.info(f"Training completed in {training_time:.2f} seconds")

except TimeoutException:
    logger.warning("Training timed out after 30 minutes, proceeding with current model state")
    # Cancel the timeout
    signal.alarm(0)
except Exception as e:
    logger.error(f"Error during training: {e}")
    # Cancel the timeout
    signal.alarm(0)
    # Save model if possible before exiting
    try:
        model.save("emergency_save_model.keras")  # Using .keras extension
        logger.info("Model saved despite training error")
    except:
        logger.error("Could not save model after error")
    raise

# Step 9.10: Define inference models for translation
logger.info("\n--- Creating Inference Models ---")

# Encoder inference model
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder inference model
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_embedding, initial_state=decoder_states_inputs
)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states
)

# Step 9.11: Define translation function with timeout protection
def translate_text(input_text, max_length=max_len_target, timeout_seconds=10):
    """Translate text using the trained model with timeout protection"""

    # Set timeout handler
    signal.signal(signal.SIGALRM, timeout_handler)
    signal.alarm(timeout_seconds)

    try:
        # Preprocess input text
        input_text = input_text.lower().strip()

        # Convert to sequence
        input_seq = source_tokenizer.texts_to_sequences([input_text])
        input_seq = pad_sequences(input_seq, maxlen=max_len_source, padding='post')

        # Encode the input sequence
        states_value = encoder_model.predict(input_seq, verbose=0)

        # Generate empty target sequence of length 1
        target_seq = np.zeros((1, 1))
        # First token is the start token (we'll use index 1 as start)
        target_seq[0, 0] = 1

        # Sampling loop
        stop_condition = False
        decoded_sentence = ''

        while not stop_condition:
            output_tokens, h, c = decoder_model.predict(
                [target_seq] + states_value, verbose=0
            )

            # Sample a token
            sampled_token_index = np.argmax(output_tokens[0, 0, :])
            sampled_word = ''

            for word, index in target_tokenizer.word_index.items():
                if index == sampled_token_index:
                    sampled_word = word
                    break

            if sampled_word:
                decoded_sentence += sampled_word + ' '

            # Exit condition: either hit max length or find stop token
            if sampled_word == '<end>' or len(decoded_sentence.split()) > max_length:
                stop_condition = True

            # Update the target sequence (length 1)
            target_seq = np.zeros((1, 1))
            target_seq[0, 0] = sampled_token_index

            # Update states
            states_value = [h, c]

        # Cancel the timeout
        signal.alarm(0)
        return decoded_sentence.strip()

    except TimeoutException:
        # Cancel the timeout
        signal.alarm(0)
        return "Translation timed out"
    except Exception as e:
        # Cancel the timeout
        signal.alarm(0)
        logger.error(f"Translation error: {e}")
        return f"Translation error: {str(e)}"

# Step 9.12: Save the trained model and tokenizers with error handling
logger.info("\n--- Saving Model and Tokenizers ---")
try:
    # Save the main model using the recommended format
    model.save("kikuyu_kiswahili_model.keras")
    logger.info("Main model saved successfully")

    # Save inference models
    encoder_model.save("encoder_model.keras")
    logger.info("Encoder model saved successfully")

    decoder_model.save("decoder_model.keras")
    logger.info("Decoder model saved successfully")

    # Save tokenizers
    with open('source_tokenizer.pickle', 'wb') as handle:
        pickle.dump(source_tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

    with open('target_tokenizer.pickle', 'wb') as handle:
        pickle.dump(target_tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

    logger.info("Tokenizers saved successfully")
except Exception as e:
    logger.error(f"Error saving model: {e}")
    logger.info("Attempting to continue despite saving error")

# Step 9.13: Evaluate the model on test examples with timeout protection
logger.info("\n--- Model Evaluation ---")
try:
    # Test with a few examples
    test_sentences = kikuyu_texts[:5]  # Use first 5 sentences as test

    logger.info("Translation Results:")
    logger.info("====================")

    for i, sentence in enumerate(test_sentences):
        # Set a timeout for each translation (10 seconds)
        translation = translate_text(sentence, timeout_seconds=10)

        logger.info(f"Example {i+1}:")
        logger.info(f"Source (Kikuyu): {sentence}")
        logger.info(f"Translation (Kiswahili): {translation}")
        logger.info(f"Reference (Kiswahili): {kiswahili_texts[i]}")
        logger.info("---")

        # Add a small delay to prevent resource exhaustion
        time.sleep(0.5)

    # Test with some individual words/phrases
    test_words = [
        "Ngai",  # God
        "mũthenya",  # day
        "thĩ",  # earth
        "maaĩ",  # water
        "mũndũ"   # person
    ]

    logger.info("\nIndividual Word Translation:")
    logger.info("===========================")

    for word in test_words:
        translation = translate_text(word, timeout_seconds=5)
        logger.info(f"Kikuyu: {word} → Kiswahili: {translation}")

except Exception as e:
    logger.error(f"Error during evaluation: {e}")
    logger.info("Evaluation phase encountered errors but process will complete")

logger.info("\n--- Translation model training and evaluation completed successfully ---")
logger.info(f"Total execution time: {time.time() - start_time:.2f} seconds")
logger.info("You can now use the saved models for translation tasks")

# Final cleanup to ensure process completes
gc.collect()

In [ ]:
# Step 9: Memory-Optimized Kikuyu-Kiswahili Translation Model Training (Fixed)

import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import gc
import os
from sklearn.model_selection import train_test_split

print("TensorFlow version:", tf.__version__)

# Force CPU usage to avoid CUDA errors
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'  # Force CPU usage

# Enable memory growth to prevent TensorFlow from allocating all GPU memory at once
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

print("Using CPU for training (mixed precision disabled)")

# Step 9.1: Load data with memory constraints
print("\n--- Loading Data ---")

# Set a limit on the number of sentences to process
MAX_SAMPLES = 1000  # Adjust based on your RAM constraints

# Load Kikuyu data
kikuyu_file_path = 'kikuyu-train-data.txt'
with open(kikuyu_file_path, 'r', encoding='utf-8') as file:
    kikuyu_lines = file.readlines()
    kikuyu_texts = [line.strip() for line in kikuyu_lines if line.strip()]
    print(f"Loaded {len(kikuyu_texts)} Kikuyu sentences")

# Load Kiswahili data
kiswahili_file_path = 'kiswahili-train-data.txt'
with open(kiswahili_file_path, 'r', encoding='utf-8') as file:
    kiswahili_lines = file.readlines()
    kiswahili_texts = [line.strip() for line in kiswahili_lines if line.strip()]
    print(f"Loaded {len(kiswahili_texts)} Kiswahili sentences")

# Ensure we have the same number of sentences in both languages
min_len = min(len(kikuyu_texts), len(kiswahili_texts), MAX_SAMPLES)
kikuyu_texts = kikuyu_texts[:min_len]
kiswahili_texts = kiswahili_texts[:min_len]
print(f"Using {min_len} parallel sentences for training")

# Print some examples to verify alignment
print("\n--- Sample Sentence Pairs ---")
for i in range(min(3, min_len)):
    print(f"Kikuyu: {kikuyu_texts[i]}")
    print(f"Kiswahili: {kiswahili_texts[i]}")
    print()

# Step 9.2: Tokenize with reduced parameters
print("\n--- Tokenizing Text ---")

# Define parameters with reduced complexity
max_num_words = 5000  # Reduced vocabulary size
max_len_source = 30   # Reduced sequence length
max_len_target = 30   # Reduced sequence length
embedding_dim = 32    # Reduced embedding dimension
latent_dim = 32       # Reduced LSTM dimension

# Create tokenizers
source_tokenizer = Tokenizer(num_words=max_num_words, oov_token='<OOV>')
target_tokenizer = Tokenizer(num_words=max_num_words, oov_token='<OOV>')

# Fit tokenizers on texts
source_tokenizer.fit_on_texts(kikuyu_texts)
target_tokenizer.fit_on_texts(kiswahili_texts)

# Calculate vocabulary sizes
source_vocab_size = min(max_num_words, len(source_tokenizer.word_index) + 1)
target_vocab_size = min(max_num_words, len(target_tokenizer.word_index) + 1)

print(f"Kikuyu vocabulary size: {source_vocab_size}")
print(f"Kiswahili vocabulary size: {target_vocab_size}")

# Convert texts to sequences
source_sequences = source_tokenizer.texts_to_sequences(kikuyu_texts)
target_sequences = target_tokenizer.texts_to_sequences(kiswahili_texts)

# Pad sequences
encoder_input_data = pad_sequences(source_sequences, maxlen=max_len_source, padding='post')
decoder_input_data = pad_sequences(target_sequences, maxlen=max_len_target, padding='post')

# Create target data (shifted by one)
decoder_target_data = np.zeros_like(decoder_input_data)
decoder_target_data[:, :-1] = decoder_input_data[:, 1:]

# Clear memory
del source_sequences, target_sequences
gc.collect()

print(f"Encoder input shape: {encoder_input_data.shape}")
print(f"Decoder input shape: {decoder_input_data.shape}")
print(f"Decoder target shape: {decoder_target_data.shape}")

# Step 9.3: Split data with smaller validation set
train_encoder_input, val_encoder_input, train_decoder_input, val_decoder_input, train_decoder_target, val_decoder_target = train_test_split(
    encoder_input_data, decoder_input_data, decoder_target_data, test_size=0.1, random_state=42
)

# Clear memory
del encoder_input_data, decoder_input_data, decoder_target_data
gc.collect()

print(f"Training samples: {len(train_encoder_input)}")
print(f"Validation samples: {len(val_encoder_input)}")

# Step 9.4: Create TensorFlow datasets for memory-efficient training
BATCH_SIZE = 8  # Reduced batch size

# Create datasets
train_dataset = tf.data.Dataset.from_tensor_slices(
    ((train_encoder_input, train_decoder_input), train_decoder_target)
).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

val_dataset = tf.data.Dataset.from_tensor_slices(
    ((val_encoder_input, val_decoder_input), val_decoder_target)
).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

# Clear memory
del train_encoder_input, train_decoder_input, train_decoder_target
del val_encoder_input, val_decoder_input, val_decoder_target
gc.collect()

# Step 9.5: Define a simpler model architecture
print("\n--- Defining Model Architecture ---")
# Encoder
encoder_inputs = Input(shape=(max_len_source,), name='encoder_inputs')
encoder_embedding = Embedding(input_dim=source_vocab_size, output_dim=embedding_dim, name='encoder_embedding')(encoder_inputs)
# Use implementation=1 for CPU-friendly LSTM
encoder_lstm = LSTM(latent_dim, return_state=True, name='encoder_lstm', implementation=1)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
encoder_states = [state_h, state_c]
# Decoder
decoder_inputs = Input(shape=(max_len_target,), name='decoder_inputs')
decoder_embedding = Embedding(input_dim=target_vocab_size, output_dim=embedding_dim, name='decoder_embedding')(decoder_inputs)
# Use implementation=1 for CPU-friendly LSTM
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True, name='decoder_lstm', implementation=1)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(target_vocab_size, activation='softmax', name='decoder_dense')
decoder_outputs = decoder_dense(decoder_outputs)
# Define the model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
# Use a more memory-efficient optimizer
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)
# Print model summary
model.summary()
# Step 9.6: Train with checkpoints and early stopping
print("\n--- Training the Model ---")
# Create checkpoint directory
checkpoint_dir = './training_checkpoints'
os.makedirs(checkpoint_dir, exist_ok=True)
# Define callbacks
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=os.path.join(checkpoint_dir, 'ckpt_{epoch}.weights.h5'),
    save_weights_only=True,
    save_best_only=True,
    monitor='val_loss',
    verbose=1
)
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)
# Reduce epochs further
EPOCHS = 5  # Reduced from 10 to 5
try:
    # Train the model
    history = model.fit(
        train_dataset,
        epochs=EPOCHS,
        validation_data=val_dataset,
        callbacks=[checkpoint_callback, early_stopping],
        verbose=1
    )
    print("\nModel training finished successfully.")
    # Save the model in the newer .keras format
    model.save('kikuyu_kiswahili_translation_model.keras')
    print("Model saved successfully in .keras format.")
except Exception as e:
    print(f"Training error: {e}")
    # Try to save the model even if training was interrupted
    try:
        model.save('kikuyu_kiswahili_translation_model_partial.keras')
        print("Partial model saved in .keras format.")
    except:
        print("Could not save partial model.")
# Step 9.7: Create inference models for translation - FIXED to avoid 'KerasTensor' object is not callable error
try:
    # Create a new encoder model for inference
    encoder_model = Model(inputs=encoder_inputs, outputs=encoder_states)
    # Create a new decoder model for inference
    decoder_state_input_h = Input(shape=(latent_dim,))
    decoder_state_input_c = Input(shape=(latent_dim,))
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    # Create new embedding and LSTM layers for the decoder model
    decoder_embedding_layer = Embedding(input_dim=target_vocab_size, output_dim=embedding_dim)
    decoder_lstm_layer = LSTM(latent_dim, return_sequences=True, return_state=True, implementation=1)
    decoder_dense_layer = Dense(target_vocab_size, activation='softmax')
    # Apply the embedding layer
    decoder_embedding_outputs = decoder_embedding_layer(decoder_inputs)
    # Apply the LSTM layer
    decoder_outputs2, state_h2, state_c2 = decoder_lstm_layer(
        decoder_embedding_outputs, initial_state=decoder_states_inputs
    )
    decoder_states2 = [state_h2, state_c2]
    # Apply the dense layer
    decoder_outputs2 = decoder_dense_layer(decoder_outputs2)
    # Create the decoder model
    decoder_model = Model(
        inputs=[decoder_inputs] + decoder_states_inputs,
        outputs=[decoder_outputs2] + decoder_states2
    )
    # Save inference models in .keras format
    encoder_model.save('kikuyu_kiswahili_encoder_model.keras')
    decoder_model.save('kikuyu_kiswahili_decoder_model.keras')
    print("Inference models saved successfully in .keras format.")
    # Save tokenizers for later use
    import pickle
    with open('source_tokenizer.pickle', 'wb') as handle:
        pickle.dump(source_tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    with open('target_tokenizer.pickle', 'wb') as handle:
        pickle.dump(target_tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    print("Tokenizers saved successfully.")
    # FIXED: Translation function that addresses tf.function retracing warnings and repetitive outputs
    # Define the translation function outside of the loop to avoid retracing
    @tf.function(reduce_retracing=True)
    def predict_step(input_seq, states_value, target_seq):
        # Returns the prediction and states
        return decoder_model([target_seq] + states_value)
    def translate_sentence(input_text):
        # Tokenize and pad the input text
        input_seq = source_tokenizer.texts_to_sequences([input_text])
        input_seq = pad_sequences(input_seq, maxlen=max_len_source, padding='post')
        # Encode the input sequence to get the internal states
        states_value = encoder_model.predict(input_seq, verbose=0)
        # Generate empty target sequence of length 1 with first token
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = 1  # Use index 1 as the start token (usually the most common word)
        # Output sequence
        decoded_sentence = ''
        # Add maximum length check and repetition detection
        max_output_length = 15  # Reduced from 30 to prevent long repetitive sequences
        # Sampling loop
        stop_condition = False
        word_count = 0
        previous_words = []
        used_tokens = set()  # Track used tokens to prevent repetition
        while not stop_condition:
            # Convert to tensors for tf.function
            target_seq_tensor = tf.convert_to_tensor(target_seq, dtype=tf.float32)
            states_value_tensors = [tf.convert_to_tensor(s, dtype=tf.float32) for s in states_value]
            # Predict next token and states
            output_tokens, h, c = predict_step(input_seq, states_value_tensors, target_seq_tensor)
            # Convert back from tensors
            output_tokens = output_tokens.numpy()
            h = h.numpy()
            c = c.numpy()
            # Sample a token - use temperature sampling to increase diversity
            temperature = 1.0
            output_tokens = output_tokens[0, -1, :] / temperature
            exp_output_tokens = np.exp(output_tokens - np.max(output_tokens))
            output_tokens = exp_output_tokens / np.sum(exp_output_tokens)
            # Apply diversity penalty for tokens we've already used
            for token in used_tokens:
                if token < len(output_tokens):
                    output_tokens[token] *= 0.7  # Reduce probability of reusing tokens
            # Sample from the distribution
            sampled_token_index = np.argmax(output_tokens)
            # Convert token to word
            sampled_word = ''
            for word, index in target_tokenizer.word_index.items():
                if index == sampled_token_index:
                    sampled_word = word
                    break
            # Exit condition: either hit max length or find end token or empty word
            if (word_count >= max_output_length or sampled_word == '' or
                sampled_token_index == 0):  # 0 is usually padding
                stop_condition = True
            else:
                # Skip OOV token in output
                if sampled_word != '<OOV>':
                    # Add space before word except for first word
                    if word_count > 0:
                        decoded_sentence += ' '
                    decoded_sentence += sampled_word
                    word_count += 1
                    previous_words.append(sampled_word)
                    used_tokens.add(sampled_token_index)
                    # Check for repetition - if last 3 words are the same, stop
                    if len(previous_words) >= 3:
                        if len(set(previous_words[-3:])) == 1:
                            stop_condition = True
            # Update the target sequence (length 1)
            target_seq = np.zeros((1, 1))
            target_seq[0, 0] = sampled_token_index
            # Update states
            states_value = [h, c]
        return decoded_sentence
    # Test the translation with a sample
    print("\n--- Testing Translation ---")
    test_sentence = kikuyu_texts[0]
    print(f"Kikuyu: {test_sentence}")
    translation = translate_sentence(test_sentence)
    print(f"Translated to Kiswahili: {translation}")
except Exception as e:
    print(f"Error creating inference models: {e}")
    print("Detailed error information:")
    import traceback
    traceback.print_exc()
print("\nTranslation model setup complete!")

In [ ]:
# Step 10: Load and Test the Trained Model (Fixed)
import tensorflow as tf
import numpy as np
import pickle
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
import logging
import time
import os
import glob

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

print(f"TensorFlow version: {tf.__version__}")
print("\n--- Loading Trained Models ---")

# First, let's find where the models are actually saved
def find_model_files():
    """Find model files in the current directory and subdirectories"""
    model_files = {
        'main_model': None,
        'encoder_model': None,
        'decoder_model': None,
        'source_tokenizer': None,
        'target_tokenizer': None
    }

    # Look for .keras or .h5 files
    keras_files = glob.glob("*.keras") + glob.glob("*.h5")
    pickle_files = glob.glob("*.pickle")

    print(f"Found {len(keras_files)} model files and {len(pickle_files)} pickle files")

    # Try to identify which file is which
    for file in keras_files:
        if "kikuyu_kiswahili_model" in file or "main_model" in file:
            model_files['main_model'] = file
        elif "encoder" in file:
            model_files['encoder_model'] = file
        elif "decoder" in file:
            model_files['decoder_model'] = file

    for file in pickle_files:
        if "source" in file or "kikuyu" in file:
            model_files['source_tokenizer'] = file
        elif "target" in file or "kiswahili" in file:
            model_files['target_tokenizer'] = file

    return model_files

# Find model files
model_files = find_model_files()
print("Found model files:", model_files)

# Global variables to store models and tokenizers
main_model = None
encoder_model = None
decoder_model = None
source_tokenizer = None
target_tokenizer = None

# Step 10.1: Load the saved models with progress indicators and better error handling
try:
    # Try to load main model
    print("Loading main model...")
    if model_files['main_model']:
        main_model = load_model(model_files['main_model'])
        print(f"✓ Main model loaded successfully from {model_files['main_model']}")
    else:
        # Try default names
        try:
            main_model = load_model("kikuyu_kiswahili_model.keras")
            print("✓ Main model loaded successfully from kikuyu_kiswahili_model.keras")
        except:
            try:
                main_model = load_model("best_model.keras")
                print("✓ Main model loaded successfully from best_model.keras")
            except:
                print("⚠ Could not load main model, but will continue with encoder/decoder models")

    # Try to load encoder model
    print("Loading encoder model...")
    if model_files['encoder_model']:
        encoder_model = load_model(model_files['encoder_model'])
        print(f"✓ Encoder model loaded successfully from {model_files['encoder_model']}")
    else:
        # Try default name
        encoder_model = load_model("encoder_model.keras")
        print("✓ Encoder model loaded successfully from encoder_model.keras")

    # Try to load decoder model
    print("Loading decoder model...")
    if model_files['decoder_model']:
        decoder_model = load_model(model_files['decoder_model'])
        print(f"✓ Decoder model loaded successfully from {model_files['decoder_model']}")
    else:
        # Try default name
        decoder_model = load_model("decoder_model.keras")
        print("✓ Decoder model loaded successfully from decoder_model.keras")

    # Load tokenizers
    print("Loading tokenizers...")
    if model_files['source_tokenizer']:
        with open(model_files['source_tokenizer'], 'rb') as handle:
            source_tokenizer = pickle.load(handle)
        print(f"✓ Source tokenizer loaded successfully from {model_files['source_tokenizer']}")
    else:
        # Try default name
        with open('source_tokenizer.pickle', 'rb') as handle:
            source_tokenizer = pickle.load(handle)
        print("✓ Source tokenizer loaded successfully from source_tokenizer.pickle")

    if model_files['target_tokenizer']:
        with open(model_files['target_tokenizer'], 'rb') as handle:
            target_tokenizer = pickle.load(handle)
        print(f"✓ Target tokenizer loaded successfully from {model_files['target_tokenizer']}")
    else:
        # Try default name
        with open('target_tokenizer.pickle', 'rb') as handle:
            target_tokenizer = pickle.load(handle)
        print("✓ Target tokenizer loaded successfully from target_tokenizer.pickle")

except Exception as e:
    print(f"Error loading models: {e}")
    print("\nTrying to recreate models from training data...")

    # If we can't load the models, let's try to recreate a simple model for demonstration
    try:
        # Load Kikuyu data
        with open('kikuyu-train-data.csv', 'r', encoding='utf-8') as f:
            kikuyu_texts = [line.strip() for line in f.readlines()]

        # Load or create Kiswahili data
        try:
            with open('kiswahili-train-data.csv', 'r', encoding='utf-8') as f:
                kiswahili_texts = [line.strip() for line in f.readlines()]
        except:
            # If no Kiswahili data, create dummy translations
            kiswahili_texts = [f"Tafsiri ya {text[:20]}..." for text in kikuyu_texts]

        # Create simple tokenizers
        from tensorflow.keras.preprocessing.text import Tokenizer

        # Limit to first 1000 sentences to save memory
        max_samples = min(1000, len(kikuyu_texts))
        kikuyu_texts = kikuyu_texts[:max_samples]
        kiswahili_texts = kiswahili_texts[:max_samples]

        # Create tokenizers
        source_tokenizer = Tokenizer(oov_token='<OOV>')
        source_tokenizer.fit_on_texts(kikuyu_texts)

        target_tokenizer = Tokenizer(oov_token='<OOV>')
        target_tokenizer.fit_on_texts(kiswahili_texts)

        print("✓ Created new tokenizers from training data")

        # Create dummy translation function
        def dummy_translate(text):
            common_translations = {
                "ngai": "mungu",
                "mũndũ": "mtu",
                "maaĩ": "maji",
                "thĩ": "dunia",
                "mũthenya": "siku"
            }

            text = text.lower()
            if text in common_translations:
                return common_translations[text]
            else:
                return "Tafsiri haiwezekani kwa sasa"

        # Set global flag to use dummy translation
        use_dummy_translation = True
        print("✓ Created simplified translation function")

    except Exception as e:
        print(f"Error creating simplified model: {e}")
        print("Cannot proceed with translation. Please run the training step again.")
        exit(1)

# Create a dictionary of common Kikuyu-Kiswahili translations
# This will be used as a fallback when the model fails to translate
common_translations = {
    # Basic religious terms
    "ngai": "mungu",
    "mwathani": "bwana",
    "kristũ": "kristo",
    "jesũ": "yesu",
    "roho mutheru": "roho mtakatifu",
    "mũtũmwo": "mtume",
    "mũnabii": "nabii",
    "kĩrĩĩkanĩro": "agano",
    "mũthĩnjĩri ngai": "kuhani",
    "igongoona": "sadaka",
    "kũhoroherio": "kusamehewa",
    "mũtharaba": "msalaba",
    "kiambiriria": "mwanzo",

    # People and family
    "mũndũ": "mtu",
    "mũndũ mũrũme": "mwanamume",
    "mũndũ wa nja": "mwanamke",
    "kaana": "mtoto",
    "kahĩĩ": "mvulana",
    "kairĩĩtu": "msichana",
    "mũtumia": "mke",
    "mũthuuriwe": "mume",
    "ithe": "baba",
    "nyina": "mama",
    "mũrũ": "mwana",
    "mwarĩ": "binti",
    "mũkũrũ": "mkubwa",
    "mũnyiinyi": "mdogo",
    "mũciĩ": "nyumba",
    "andũ": "watu",
    "mũrĩithi": "mchungaji",
    "mũrĩmi": "mkulima",
    "mũthamaki": "mfalme",
    "mũtongoria": "kiongozi",
    "mũthuuri": "mzee",
    "mũtumia mũkũrũ": "bibi",

    # Body parts
    "mũtwe": "kichwa",
    "maitho": "macho",
    "matũ": "masikio",
    "iniũrũ": "pua",
    "kanua": "kinywa",
    "rũrĩmĩ": "ulimi",
    "magego": "meno",
    "ngingo": "shingo",
    "guoko": "mkono",
    "ciara": "vidole",
    "nda": "tumbo",
    "magũrũ": "miguu",
    "thakame": "damu",
    "ngoro": "moyo",
    "mwĩrĩ": "mwili",
    "gĩthũri": "uso",

    # Nature and environment
    "thĩ": "dunia",
    "iguru": "mbingu",
    "riũa": "jua",
    "mweri": "mwezi",
    "njata": "nyota",
    "maaĩ": "maji",
    "rũũĩ": "mto",
    "iria": "bahari",
    "mũtĩ": "mti",
    "mũgũnda": "shamba",
    "nyeki": "nyasi",
    "kĩrĩma": "mlima",
    "werũ": "jangwa",
    "mbura": "mvua",
    "rũhuuho": "upepo",
    "mũkũngambura": "upinde",
    "ũtheri": "nuru",
    "nduma": "giza",
    "mwaki": "moto",
    "mũhu": "majivu",
    "tĩĩri": "ardhi",
    "mahiga": "mawe",
    "rũkũngũ": "vumbi",

    # Animals
    "nyamũ": "mnyama",
    "nyoni": "ndege",
    "thamaki": "samaki",
    "ng'ombe": "ng'ombe",
    "ndeegwa": "dume",
    "mbũri": "mbuzi",
    "ng'ondu": "kondoo",
    "ngũkũ": "kuku",
    "mbwa": "mbwa",
    "nyoka": "nyoka",
    "ndutuura": "njiwa",
    "ihuru": "kunguru",
    "njũũi": "nyuki",
    "rwagi": "chui",
    "njogu": "ndovu",
    "ngamĩĩra": "ngamia",
    "njagĩ": "nyani",

    # Food and plants
    "irio": "chakula",
    "mũgate": "mkate",
    "ngano": "ngano",
    "mbembe": "mahindi",
    "mũcũngwa": "chungwa",
    "matunda": "matunda",
    "mbegu": "mbegu",
    "maguta": "mafuta",
    "cumbĩ": "chumvi",
    "ũbaani": "ubani",
    "ndibei": "divai",
    "mũtamaiyũ": "mzeituni",
    "mĩthabibũ": "mizabibu",
    "mũtu": "unga",

    # Time and numbers
    "mũthenya": "siku",
    "utukũ": "usiku",
    "rũciinĩ": "asubuhi",
    "mũthenya": "mchana",
    "hwaĩinĩ": "jioni",
    "ũmũthĩ": "leo",
    "rũciũ": "kesho",
    "ira": "jana",
    "kĩambĩrĩria": "mwanzo",
    "mũthia": "mwisho",
    "mweri": "mwezi",
    "mwaka": "mwaka",
    "ihinda": "wakati",
    "ũmwe": "moja",
    "eerĩ": "mbili",
    "atatũ": "tatu",
    "ana": "nne",
    "ithaano": "tano",
    "ithathatũ": "sita",
    "mũgwanja": "saba",
    "inyaanya": "nane",
    "kenda": "tisa",
    "ikũmi": "kumi",
    "mĩrongo ĩĩrĩ": "ishirini",
    "mĩrongo ĩtatũ": "thelathini",
    "mĩrongo ĩna": "arubaini",
    "mĩrongo ĩtaano": "hamsini",
    "mĩrongo ĩtandatũ": "sitini",
    "mĩrongo mũgwanja": "sabini",
    "mĩrongo ĩnaana": "themanini",
    "mĩrongo kenda": "tisini",
    "igana": "mia",
    "ngiri": "elfu",

    # Places and buildings
    "itũũra": "mji",
    "njĩra": "njia",
    "hema": "hema",
    "kĩgongoona": "madhabahu",
    "hema ya gũtũnganwo": "hema la mkutano",
    "mũromo": "mlango",
    "ndirica": "dirisha",

    # Biblical names and places
    "adamu": "adamu",
    "habili": "abeli",
    "kaini": "kaini",
    "noa": "noa",
    "thabina": "safina",
    "musa": "musa",
    "harũni": "aroni",
    "joshua": "yoshua",
    "jakubu": "yakobo",
    "josefu": "yosefu",
    "misiri": "misri",
    "jerusalemu": "yerusalemu",
    "bethilehemu": "bethlehemu",
    "herode": "herode",
    "paũlũ": "paulo",
    "isiraeli": "israeli",
    "juda": "yuda",
    "sinai": "sinai",
    "yordani": "jorodani",
    "edeni": "edeni",
    "ararati": "ararati",
    "farati": "frati",
    "babeli": "babeli",

    # Verbs and actions
    "kũruta": "kutoa",
    "kũrĩa": "kula",
    "kũnyua": "kunywa",
    "gũkoma": "kulala",
    "gũciara": "kuzaa",
    "gũtũũra": "kuishi",
    "gũkua": "kufa",
    "kũũraga": "kuua",
    "kũhanda": "kupanda",
    "kũgetha": "kuvuna",
    "gũthiĩ": "kwenda",
    "gũũka": "kuja",
    "kũmenya": "kujua",
    "kwĩra": "kusema",
    "kũigua": "kusikia",
    "kuona": "kuona",
    "kũraathima": "kubariki",
    "kũruma": "kulaani",
    "gũthaathaiya": "kuabudu",
    "kũhingũra": "kufungua",
    "kũhinga": "kufunga",
    "gũtoonya": "kuingia",
    "kũuma": "kutoka",
    "kũrĩĩkanĩra": "kuagana",
    "gũcoka": "kurudi",
    "kũingĩha": "kuongezeka",
    "kũhũa": "kupungua",
    "gũtũma": "kutuma",
    "kũruta": "kufanya",
    "kũrĩa": "kula",
    "kũnyua": "kunywa",
    "gũkoma": "kulala",
    "gũciara": "kuzaa",
    "gũtũũra": "kuishi",
    "gũkua": "kufa",

    # Adjectives and qualities
    "mũnene": "kubwa",
    "mũniini": "ndogo",
    "mũraihu": "ndefu",
    "mũkuhĩ": "fupi",
    "mũthaka": "mzuri",
    "mũũru": "mbaya",
    "mũthingu": "mwadilifu",
    "mũgiro": "najisi",
    "theru": "takatifu",
    "mũrũaru": "mgonjwa",
    "mũhoro": "mtulivu",
    "njega": "nzuri",
    "njũru": "mbaya",
    "ngũrũ": "kali",
    "nyoroku": "laini",

    # Other common words
    "wĩra": "kazi",
    "ũhoro": "habari",
    "rũũri": "alama",
    "mũiyũro": "gharika",
    "mũrango": "mlango",
    "maheeni": "uongo",
    "ũũgĩ": "hekima",
    "guoya": "hofu",
    "kĩrumi": "laana",
    "ihera": "adhabu",
    "wara": "ujanja",
    "mĩaka": "miaka",
    "mĩthenya": "siku",
    "mĩeri": "miezi",
    "mĩhĩrĩga": "makabila",
    "mbarĩ": "ukoo",
    "atongoria": "viongozi",
    "mũingĩ": "umati",
    "indo": "vitu",
    "kĩndũ": "kitu",
    "rĩĩtwa": "jina",
    "ciugo": "maneno",
    "mĩtaratara": "mpangilio",
    "njiarwa": "vizazi",
    "mĩtũũrirũ": "filimbi",
    "inanda": "zeze",
    "igera": "chuma",
    "gĩcango": "shaba",
    "mũturi": "mhunzi",
    "mũrĩmi": "mkulima",
    "mũrĩithi": "mchungaji",
    "rũũru": "kundi",
    "mahiũ": "mifugo",
    "magetha": "mavuno",
    "irigithaathi": "mzaliwa wa kwanza",
    "mũrũ wa nyina": "ndugu",
    "mwarĩ wa nyina": "dada",
    "mũtumia": "mke",
    "mũthuuriwe": "mume",
    "atumia": "wake",
    "aariũ": "wana",
    "mũrũ": "mwana",
    "mwarĩ": "binti"
}

# Step 10.2: Define improved translation function with better handling
def translate_text(input_text, max_length=30, timeout=30):
    """Translate Kikuyu text to Kiswahili with improved handling"""
    print(f"Translating: '{input_text}'")
    start_time = time.time()

    # Check if we should use dummy translation
    if 'use_dummy_translation' in globals() and use_dummy_translation:
        print(f"Using simplified translation for: '{input_text}'")
        return dummy_translate(input_text.lower())

    # Check if all required components are available
    if None in [encoder_model, decoder_model, source_tokenizer, target_tokenizer]:
        return "Translation not possible: Missing required models or tokenizers"

    # First check if the word is in our common translations dictionary
    input_lower = input_text.lower().strip()
    if input_lower in common_translations:
        return common_translations[input_lower]

    try:
        # Preprocess input text
        input_text = input_text.lower().strip()

        # Handle empty input
        if not input_text:
            return "Please enter some text to translate."

        # Handle very long input
        if len(input_text.split()) > 10:
            return "Input text too long. Please enter a shorter phrase (max 10 words)."

        # Convert to sequence
        input_seq = source_tokenizer.texts_to_sequences([input_text])

        # Check if any words were recognized
        if not any(input_seq[0]):
            # Try to find partial matches in common translations
            for key in common_translations:
                if key in input_text or input_text in key:
                    return common_translations[key] + " (partial match)"
            return "No words recognized. Please try different Kikuyu words."

        input_seq = pad_sequences(input_seq, maxlen=max_length, padding='post')

        # Encode the input sequence
        if len(encoder_model.inputs) == 1:
            # Standard encoder
            encoder_outputs, state_h, state_c = encoder_model.predict(input_seq, verbose=0)
            states_value = [state_h, state_c]
        else:
            # Alternative encoder structure
            states_value = encoder_model.predict(input_seq, verbose=0)

        # Generate empty target sequence of length 1
        target_seq = np.zeros((1, 1))
        # First token is the start token (we'll use index 1 as start)
        target_seq[0, 0] = 1

        # Sampling loop
        stop_condition = False
        decoded_sentence = ''
        generated_tokens = []
        max_tokens = 20  # Limit output length

        while not stop_condition:
            # Check timeout
            if time.time() - start_time > timeout:
                return "Translation timed out. Please try again with a shorter phrase."

            # Predict next token
            try:
                if len(decoder_model.inputs) == 3:  # Standard structure
                    output_tokens, h, c = decoder_model.predict(
                        [target_seq] + states_value, verbose=0
                    )
                    states_value = [h, c]
                else:  # Alternative structure
                    output_tokens = decoder_model.predict(
                        [target_seq] + states_value if isinstance(states_value, list) else [target_seq, states_value],
                        verbose=0
                    )
                    if isinstance(output_tokens, list):
                        h, c = output_tokens[1], output_tokens[2]
                        output_tokens = output_tokens[0]
                        states_value = [h, c]
            except Exception as e:
                print(f"Error during prediction: {e}")
                # Fall back to common translations
                for key in common_translations:
                    if key in input_text or input_text in key:
                        return common_translations[key] + " (fallback)"
                return "Error during translation process. Using fallback."

            # Sample a token
            sampled_token_index = np.argmax(output_tokens[0, 0, :])

            # Check for repetition - if we've generated this token multiple times in a row, stop
            if len(generated_tokens) >= 3 and all(t == sampled_token_index for t in generated_tokens[-3:]):
                break

            generated_tokens.append(sampled_token_index)

            sampled_word = ''
            for word, index in target_tokenizer.word_index.items():
                if index == sampled_token_index:
                    sampled_word = word
                    break

            if sampled_word:
                # Don't add the same word multiple times in a row
                words = decoded_sentence.split()
                if not words or words[-1] != sampled_word:
                    decoded_sentence += sampled_word + ' '

            # Exit conditions
            if (sampled_word == '<end>' or
                len(decoded_sentence.split()) > max_length or
                len(generated_tokens) >= max_tokens):
                stop_condition = True

            # Update the target sequence (length 1)
            target_seq = np.zeros((1, 1))
            target_seq[0, 0] = sampled_token_index

        # Clean up the output
        result = decoded_sentence.strip()

        # Remove repetitive patterns
        words = result.split()
        cleaned_words = []
        for i, word in enumerate(words):
            if i == 0 or word != words[i-1]:
                cleaned_words.append(word)

        result = ' '.join(cleaned_words)

        # If result is empty or just repetitive words, return a message
        if not result or len(set(result.split())) <= 1:
            # Try to find partial matches in common translations
            for key in common_translations:
                if key in input_text or input_text in key:
                    return common_translations[key] + " (partial match)"

            # If no partial match, use the first word of input to find a match
            input_words = input_text.split()
            if input_words:
                first_word = input_words[0]
                for key in common_translations:
                    if key.startswith(first_word) or first_word.startswith(key):
                        return common_translations[key] + " (word match)"

            return "Could not generate a meaningful translation."

        print(f"Translation completed in {time.time() - start_time:.2f} seconds")
        return result

    except Exception as e:
        print(f"Translation error: {e}")
        # Fall back to common translations
        for key in common_translations:
            if key in input_text or input_text in key:
                return common_translations[key] + " (error fallback)"
        return f"Error: {str(e)}"

# Step 10.3: Create a simple text-based translation interface (no widgets)
print("\n--- Kikuyu to Kiswahili Translator ---")
print("Enter 'q' to quit the translator")

# Define some simple test words that should work well
test_sentences = [
    "Ngai",  # God
    "mũndũ",  # person
    "maaĩ",  # water
    "thĩ",  # earth
    "mũthenya"  # day
]

print("\nTesting translation with simple words:")
for sentence in test_sentences:
    translation = translate_text(sentence)
    print(f"Kikuyu: {sentence}")
    print(f"Kiswahili: {translation}")
    print("-" * 50)

# Interactive translation loop
print("\n=== Interactive Translation Interface ===")
print("Type short Kikuyu words or phrases (1-5 words work best)")
while True:
    user_input = input("\nEnter Kikuyu text to translate (or 'q' to quit): ")

    if user_input.lower() == 'q':
        print("Exiting translator. Thank you!")
        break

    if not user_input.strip():
        print("Please enter some text to translate.")
        continue

    print("Translating...")
    translation = translate_text(user_input)
    print(f"\nKikuyu: {user_input}")
    print(f"Kiswahili: {translation}")
    print("-" * 50)

print("\nTranslation session completed.")

In [ ]:
# Add this to your Google Colab notebook
!pip install flask flask-cors pyngrok

from flask import Flask, request, jsonify
from flask_cors import CORS
import threading
from google.colab import output
import nest_asyncio

# Apply nest_asyncio to allow nested event loops
nest_asyncio.apply()

# Create Flask app
app = Flask(__name__)
CORS(app)  # Enable CORS for all routes

# Define translation endpoint that will use your existing translation function
@app.route('/translate', methods=['POST'])
def translate_api():
    data = request.json
    text = data.get('text', '')

    # Use your existing translation function
    translation = translate_text(text)

    return jsonify({'translation': translation})

# Function to run the Flask app
def run_flask():
    app.run(host='0.0.0.0', port=8000)

# Start the Flask app in a separate thread
threading.Thread(target=run_flask).start()

# Use ngrok to expose the local server to the internet
!pip install pyngrok
from pyngrok import ngrok

# Set up ngrok with authentication
# You need to sign up at https://dashboard.ngrok.com/signup
# Then get your authtoken from https://dashboard.ngrok.com/get-started/your-authtoken
ngrok_auth_token = "2woKs9KZbRrC2icpYM9eUJxbZhP_LFmZgX7nwey1z9hEU7E8"  # Replace with your actual token
ngrok.set_auth_token(ngrok_auth_token)

# Now connect to ngrok
public_url = ngrok.connect(8000)
print(f"Public URL: {public_url}")

In [ ]:
#REMEDIES OF FIXING THE TRANSLATION ISSUES .

#*In google colab.
#1.In the Google colab Runtime section, change the runtime type from CPU to GPU then after sometime change it back to CPU.

#2.Then still in the Runtime section, choose the Restart session option.

#*In NGRock
#(After signing in and going to the official dashboard)
#1. On the left side, scroll down and locate settings,click settings and scroll down and locate "Revoke sessions" button.

#2. Go back to the the main dashboard where your Auth token is and scroll down till bottom and locate "Reset Authtoken" button.

#3. Copy the new Auth token and go back to google colab and paste it where it is supposed to be placed .

#4. Finally after you have done all the above steps, Click Runtime and Click "Run All" in order to run all the google colab processes again and the issue will be fixed.

#* At the end of the process you should be provided with a public URL which you are supposed to copy it in the your VS code , Javascript section.

#* Run the live server and then use the words in the local dictionary for translation.